In [ ]:
import json, os
import cv2
import torch
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd_scorer import AKPDNormalizationTransform, AKPDScorerNetwork
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# extract sample dataset
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    select * from keypoint_annotations 
    where pen_id=61 
    and captured_at between '2019-09-13' and '2019-09-21'
    and keypoints is not null;
"""

df = rds_access_utils.extract_from_database(query)

aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

# initialize data transforms so that we can run inference with AKPD scorer network
normalize_centered_2D_transform_akpd = NormalizeCentered2D(rotate=False, center=True)
akpd_normalization_transform = AKPDNormalizationTransform()
to_tensor_transform = ToTensor()

# initialize data transforms so that we can run inference with biomass neural network
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')
akpd_scorer_network = torch.load('/root/data/alok/biomass_estimation/playground/akpd_scorer_model.pb')

In [ ]:
# confidence_scores, weight_predictions = [], []
for idx, row in df.iterrows():
    if idx < 7126:
        continue
    if idx % 100 == 0:
        print(idx)
        print(len(confidence_scores))

    # run AKPD
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
    cm = row.camera_metadata
    akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)

    # run template matching
#     left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
#     right_image_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
#     imageL = cv2.imread(left_image_f)
#     imageR = cv2.imread(right_image_f)
#     H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata)

    # run AKPD scoring network
    input_sample = {
        'keypoints': akpd_keypoints[0],
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_akpd.__call__(input_sample)

    akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
    confidence_score = akpd_scorer_network(tensorized_kps['kp_input']).item()
    print(confidence_score)
    

    # run biomass estimation
    input_sample = {
        'keypoints': akpd_keypoints[0],
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    print(weight_prediction)
    
    confidence_scores.append(confidence_score)
    weight_predictions.append(weight_prediction)

In [ ]:
print('done!')

In [ ]:
cs = np.array(confidence_scores)
wp = np.array(weight_predictions)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(wp[cs > 0.5])
plt.title('AKPD for 9/13-9/20 IMR images annotated by Cogito')
plt.xlabel('Predicted weight (g)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [ ]:
np.mean(wp[cs > 0.5])

In [ ]:
np.median(wp[cs > 0.5])

In [ ]:
(cs > 0.5).sum() / len(cs)

In [ ]:
(2261.8650970433587 - 2070) / 2070

In [ ]:
(2213.645428419113 - 2070) / 2070